### oikotie.fi Dataset Creation(w/ Python BeautifulSoup and Selenium)
Scrape a list of jobs to create a dataset 

In [25]:
from bs4 import BeautifulSoup 
import requests
import urllib
import pandas as pd
import os

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains
from tqdm import tqdm
import time

In [43]:
from selenium import webdriver    
import chromedriver_autoinstaller
from selenium.webdriver.chrome.service import Service

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"}

joblist=[]


def getJobs(job_title, location):
    getVars = {'hakusana' : job_title, 'kaupunki' : location}
    url = ('https://tyopaikat.oikotie.fi/tyopaikat?' + urllib.parse.urlencode(getVars))
    chromedriver_autoinstaller.install()
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--headless")
    from time import sleep
    driver = webdriver.Chrome(service=Service(),options=chrome_options)
    driver.get(url)
    time.sleep(3)
    while True:
        try: 
            show_more = driver.find_element(By.XPATH,"""//button[.//*[contains(text(),'Näytä lisää')] and not(@disabled)]""")
            driver.execute_script("arguments[0].click();", show_more)
            print("Show more button clicked")
            time.sleep(2)
        except:
            print("No more Show more button")
            break
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    jobs=soup.find_all("div",{"class":"job-ad-list-item-wrapper"})
    for item in jobs:
        job={
           "title" : item.find('span', class_='text-clamped title').text.strip(),
            "company" : item.find('span', class_='employer').text.strip(),
            "company" : item.find('div', class_='location').text.strip(),
            "link" : 'https://tyopaikat.oikotie.fi/' + item.a["href"],
            "date" : item.find('div', class_='end-date') .text.strip()
            }
        joblist.append(job)
        
getJobs("data","helsinki,espoo")

print(len(joblist))



Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
Show more button clicked
No more Show more button
324


In [44]:
import pandas as pd

In [45]:
df=pd.DataFrame(joblist)
print(df.head())

                         title                                 company  \
0  Senior Full Stack Developer  Työntekijä  € \n              Helsinki   
1             Data-Analyytikko                          Espoo, Uusimaa   
2                Data Engineer                                Helsinki   
3                 Data Manager                                Helsinki   
4      Data Platform Architect        Etätyö  \n              Helsinki   

                                                link                date  
0  https://tyopaikat.oikotie.fi//avoimet-tyopaika...    Sulkeutuu 09.06.  
1  https://tyopaikat.oikotie.fi//avoimet-tyopaika...    Sulkeutuu 25.05.  
2  https://tyopaikat.oikotie.fi//avoimet-tyopaika...  10 päivää jäljellä  
3  https://tyopaikat.oikotie.fi//avoimet-tyopaika...   5 päivää jäljellä  
4  https://tyopaikat.oikotie.fi//avoimet-tyopaika...    Sulkeutuu 23.06.  


In [48]:
df.to_csv("oikotiejoblist.csv",index=False)

In [51]:
df1=pd.read_csv("oikotiejoblist.csv")
print(df1.columns)


Index(['title', 'company', 'link', 'date'], dtype='object')
